In [1]:
import os
from dotenv import load_dotenv
from io import BytesIO
import pickle
import joblib
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

load_dotenv()
# Leer las variables de entorno
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')


## Funciones ha utilizar

In [2]:
def do_query(query):
    try:
        # Crear un cursor
        conn = psycopg2.connect(
                dbname="punto_seguro",
                user=DB_USER,
                password=DB_PASSWORD,
                host="pg-ed88bff-srodriguezcabana-914d.h.aivencloud.com",
                port="23178"
            )
        conn.autocommit = True
        cursor = conn.cursor()

        df_resultados = pd.read_sql_query(query, conn)
        
        cursor.close()
        return df_resultados
        
    except Exception as e:
        print(f"Error: {e}")

def subir_modelo(modelo, municipio):
    try:
        # Conectarse a PostgreSQL
        conn = psycopg2.connect(
            dbname="punto_seguro", 
            user=DB_USER, 
            password=DB_PASSWORD, 
            host="pg-ed88bff-srodriguezcabana-914d.h.aivencloud.com", 
            port="23178"
        )
        conn.autocommit = True  # Para evitar commit explícitos
        cursor = conn.cursor()

        # Serializar el modelo usando pickle
        modelo_blob = BytesIO()
        pickle.dump(modelo, modelo_blob)
        modelo_blob.seek(0)  # Asegúrate de poner el puntero al principio del archivo

        # Descripción del modelo
        descripcion = f"Predicción precipitacion"

        # Insertar el modelo serializado en la base de datos
        cursor.execute("""
            INSERT INTO ps_app_modelosprediccionprecipitacion (
                modeloblob,
                descripcion,
                municipio
            ) VALUES (%s, %s, %s)
        """, (modelo_blob.read(), descripcion, municipio))

        print(f"Modelo de predicción para {municipio} guardado correctamente.")

    except Exception as e:
        print(f"Error al guardar el modelo: {e}")

    finally:
        cursor.close()
        conn.close()



## Consulta a la DB

In [3]:
query1="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('TEMPERATURA DEL AIRE A 2 m');
"""
data_temp = do_query(query1)

/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


In [4]:
data_temp

,fechaobservacion,valorobservado,departamento,municipio
0,2023-01-10 00:02:00+00:00,13.20000,BOYACÁ,GUACAMAYAS
1,2023-03-10 01:34:00+00:00,29.40000,ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA...,SAN ANDRÉS
2,2023-03-10 01:36:00+00:00,28.10000,MAGDALENA,SANTA MARTA
3,2023-03-10 01:36:00+00:00,29.30000,ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA...,SAN ANDRÉS
4,2023-03-10 01:36:00+00:00,28.30000,BOLIVAR,CARTAGENA DE INDIAS
...,...,...,...,...
5110663,2024-10-09 02:00:00+00:00,20.40000,ANTIOQUIA,BELLO
5110664,2024-10-09 02:00:00+00:00,11.00000,BOYACÁ,GUACAMAYAS
5110665,2024-10-09 02:00:00+00:00,14.84049,CUNDINAMARCA,TENA
5110666,2024-10-09 02:00:00+00:00,4.50000,BOYACÁ,SOCOTÁ


In [5]:
query2="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('PRECIPITACIÓN');
"""
data_precipitacion = do_query(query2)
data_precipitacion

/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


,fechaobservacion,valorobservado,departamento,municipio
0,2023-01-10 00:01:00+00:00,0.0,CUNDINAMARCA,GUAYABETAL
1,2023-02-10 03:30:00+00:00,0.0,CUNDINAMARCA,GUTIÉRREZ
2,2023-02-10 03:30:00+00:00,0.0,HUILA,SUAZA
3,2023-02-10 03:30:00+00:00,0.0,CUNDINAMARCA,ANOLAIMA
4,2023-02-10 03:30:00+00:00,0.0,CAUCA,GUAPI
...,...,...,...,...
15518423,2024-07-11 23:59:00+00:00,0.0,BOGOTÁ,BOGOTA D.C
15518424,2024-07-11 23:59:00+00:00,0.0,BOGOTÁ,BOGOTA D.C
15518425,2024-07-11 23:59:00+00:00,0.0,BOGOTÁ,BOGOTA D.C
15518426,2024-07-11 23:59:00+00:00,0.0,CUNDINAMARCA,GUAYABETAL


In [6]:
query3="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('DIRECCIÓN DEL VIENTO');
"""
data_direviento = do_query(query3)
data_direviento

/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


,fechaobservacion,valorobservado,departamento,municipio
0,2023-10-04 20:40:00+00:00,4.0,CUNDINAMARCA,GIRARDOT
1,2023-10-01 13:30:00+00:00,111.0,ATLANTICO,BARRANQUILLA
2,2023-10-01 13:30:00+00:00,160.0,CESAR,VALLEDUPAR
3,2023-10-01 13:30:00+00:00,253.0,CHOCO,CÉRTEGUI
4,2023-10-01 13:30:00+00:00,278.0,CHOCO,CÉRTEGUI
...,...,...,...,...
3888425,2024-07-11 15:50:00+00:00,282.0,LA GUAJIRA,URUMITA
3888426,2024-07-11 15:50:00+00:00,62.0,NARIÑO,IMUÉS
3888427,2024-07-11 15:50:00+00:00,265.0,NARIÑO,CONTADERO
3888428,2024-07-11 15:50:00+00:00,227.0,BOYACÁ,CUÍTIVA


In [7]:
query4="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('GPRS - HUMEDAD DEL AIRE A 2 m');
"""
data_humedad = do_query(query4)
data_humedad

/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


,fechaobservacion,valorobservado,departamento,municipio
0,2023-02-10 14:52:00+00:00,66.0,CESAR,VALLEDUPAR
1,2023-02-10 14:52:00+00:00,44.0,ANTIOQUIA,RIONEGRO
2,2023-02-10 14:52:00+00:00,29.0,BOYACÁ,GUACAMAYAS
3,2023-02-10 14:52:00+00:00,100.0,RISARALDA,PEREIRA
4,2023-02-10 14:52:00+00:00,81.0,BOLIVAR,CARTAGENA DE INDIAS
...,...,...,...,...
3979478,2024-02-11 19:40:00+00:00,83.0,CESAR,VALLEDUPAR
3979479,2024-02-11 19:40:00+00:00,56.0,NORTE DE SANTANDER,CÚCUTA
3979480,2024-02-11 19:40:00+00:00,58.0,AMAZONAS,LETICIA
3979481,2024-02-11 19:40:00+00:00,27.0,NARIÑO,ALDANA


In [8]:
query5="""
SELECT   
    o.fechaobservacion, 
    o.valorobservado, 
    e.departamento, 
    e.municipio 
FROM 
    ps_app_observations o
JOIN 
    ps_app_stations e ON o.codigoestacion = e.codigoestacion
JOIN 
    ps_app_sensors s ON o.codigosensor = s.codigosensor
WHERE 
    s.descripcionsensor IN ('PRESIÓN ATMOSFÉRICA');
"""
data_presion = do_query(query5)
data_presion

/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


,fechaobservacion,valorobservado,departamento,municipio
0,2023-01-10 00:02:00+00:00,852.4,ANTIOQUIA,MEDELLÍN
1,2023-03-10 15:42:00+00:00,787.3,ANTIOQUIA,RIONEGRO
2,2023-03-10 15:42:00+00:00,1005.7,CORDOBA,MONTERÍA
3,2023-03-10 15:42:00+00:00,978.4,CESAR,VALLEDUPAR
4,2023-03-10 15:42:00+00:00,1007.3,ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA...,SAN ANDRÉS
...,...,...,...,...
4544443,2024-10-28 04:12:00+00:00,758.0,BOYACÁ,SOGAMOSO
4544444,2024-10-28 04:12:00+00:00,902.9,VALLE DEL CAUCA,PALMIRA
4544445,2024-10-28 04:12:00+00:00,998.3,AMAZONAS,LETICIA
4544446,2024-10-28 04:12:00+00:00,871.3,QUINDÍO,ARMENIA


## Entrenamiento

In [9]:
# Convertir fecha a formato datetime
for df in [data_temp, data_precipitacion, data_direviento, data_humedad, data_presion]:
    df['fechaobservacion'] = pd.to_datetime(df['fechaobservacion'])


municipios = do_query('SELECT DISTINCT municipio FROM ps_app_stations;')
municipios = municipios['municipio'].to_numpy()

for municipio in municipios:

    df_temp = data_temp[data_temp['municipio'] == municipio]
    df_prec = data_precipitacion[data_precipitacion['municipio'] == municipio]
    df_dir_viento = data_direviento[data_direviento['municipio'] == municipio]
    df_humedad = data_humedad[data_humedad['municipio'] == municipio]
    df_presion = data_presion[data_presion['municipio'] == municipio]

    # Unir los dataframes por fecha
    df = pd.merge(df_temp[['fechaobservacion', 'valorobservado']], df_prec[['fechaobservacion', 'valorobservado']], on='fechaobservacion', how='inner', suffixes=('_temp', '_prec'))
    df = pd.merge(df, df_dir_viento[['fechaobservacion', 'valorobservado']], on='fechaobservacion', how='inner')
    df = pd.merge(df, df_humedad[['fechaobservacion', 'valorobservado']], on='fechaobservacion', how='inner')
    df = pd.merge(df, df_presion[['fechaobservacion', 'valorobservado']], on='fechaobservacion', how='inner')

    # Renombrar columnas para mayor claridad
    df.columns = ['fechaobservacion', 'temperatura', 'precipitacion', 'direccion_viento', 'humedad', 'presion']

    # Eliminar filas con valores nulos
    #df.dropna(inplace=True)

    # Verificar si el DataFrame no está vacío
    if df.empty:
        print(f"No hay datos suficientes para {municipio}. Saltando.")
        continue

    # Separar variables predictoras (X) y variable objetivo (y)
    X = df[['temperatura', 'direccion_viento', 'humedad', 'presion']]
    y = df['precipitacion']

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Crear y entrenar el modelo
    modelo = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)

    # Subir el modelo a la base de datos
    subir_modelo(modelo, municipio)

    # Realizar predicciones
    y_pred = modelo.predict(X_test)

    # Evaluar el modelo
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Error Absoluto Medio (MAE): {mae:.2f}")

    # Mostrar predicciones vs valores reales
    resultados = pd.DataFrame({'Real': y_test, 'Predicción': y_pred})
    print(resultados.head())


/tmp/ipykernel_4752/2335507201.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql_query(query, conn)


No hay datos suficientes para ANORÍ. Saltando.
No hay datos suficientes para PASCA. Saltando.
No hay datos suficientes para OIBA. Saltando.
No hay datos suficientes para DABEIBA. Saltando.
No hay datos suficientes para SOCORRO. Saltando.
Modelo de predicción para CARTAGENA DE INDIAS guardado correctamente.
Error Absoluto Medio (MAE): 0.03
        Real  Predicción
107542   0.0         0.0
101098   0.0         0.0
98666    0.0         0.0
92402    0.0         0.0
48732    0.0         0.0
No hay datos suficientes para SUAITA. Saltando.
No hay datos suficientes para AYAPEL. Saltando.
No hay datos suficientes para ABRIAQUÍ. Saltando.
No hay datos suficientes para TIBÚ. Saltando.
No hay datos suficientes para GUACAMAYAS. Saltando.
No hay datos suficientes para QUEBRADANEGRA. Saltando.
No hay datos suficientes para TABIO. Saltando.
No hay datos suficientes para VALLE DE SAN JUAN. Saltando.
No hay datos suficientes para TUNJA. Saltando.
No hay datos suficientes para SOACHA. Saltando.
No hay da